In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


### 导入数据

In [28]:
csv_path = 'sqlResult_1558435.csv'

news = pd.read_csv(csv_path, encoding='gb18030')

In [29]:
news.head(5)

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [30]:
len(news)

89611

### 数据预处理

drop null

In [31]:
type(news)

pandas.core.frame.DataFrame

subset : array-like, optional
Labels along other axis to consider, e.g. if you are dropping rows these would be a list of columns to include.
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html

In [32]:
news_dropna = news.dropna(subset=['source', 'content'])

In [33]:
print('删掉了{}个source or content有空值的行'.format(len(news) - len(news_dropna)))

删掉了2559个source or content有空值的行


Consider only two features 'source' and 'content'. Dropping all other features.

In [34]:
data = news_dropna.loc[:,['content', 'source']]

In [35]:
data.head()

,content,source
0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,快科技@http://www.kkj.cn/
1,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,快科技@http://www.kkj.cn/
2,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,快科技@http://www.kkj.cn/
3,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,新华社
4,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,深圳大件事


Label Encoding

In [36]:
data[data['source'] == '新华社']['source'].value_counts()

新华社    78661
Name: source, dtype: int64

In [37]:
data[data['source'] != '新华社']['source'].value_counts()[:5]

中国新闻网        523
中国证券报?中证网    510
参考消息网        385
环球网          308
南方日报第01版     270
Name: source, dtype: int64

In [40]:
# 添加一列ori_source, 方便识别后续具体某一文章来源
data.loc[:,'ori_source'] = data['source'].values.tolist()
data[:3]

,content,source,ori_source
0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,快科技@http://www.kkj.cn/,快科技@http://www.kkj.cn/
1,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,快科技@http://www.kkj.cn/,快科技@http://www.kkj.cn/
2,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,快科技@http://www.kkj.cn/,快科技@http://www.kkj.cn/


In [41]:
# let the article which source is '新华社' be the value 0.

pos_index = data[data.loc[:,'source']=='新华社'].index.tolist()
neg_index = data[data.loc[:,'source']!='新华社'].index.tolist()
data.loc[pos_index,'source'] = 1
data.loc[neg_index,'source'] = 0
# A value is trying to be set on a copy of a slice from a DataFrame.
# Try using .loc[row_indexer,col_indexer] = value instead


In [42]:
data[data.loc[:,'source']=='新华社'].index.tolist()

[]

In [43]:
data[:5]

,content,source,ori_source
0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,0,快科技@http://www.kkj.cn/
1,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,0,快科技@http://www.kkj.cn/
2,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,0,快科技@http://www.kkj.cn/
3,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,1,新华社
4,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,0,深圳大件事


,content,source,ori_source
0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,0,1
1,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,0,1
2,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,0,1


# Model Construction

In [164]:
import jieba
import re

from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer

In [110]:
type(data['content'])

pandas.core.series.Series

### 1.Training Data and Labels Construction

In [125]:
# 选取数据的三种方式
s1 = data.content
s2 = data['content']
s3 = data.loc[:,'content']
[print(type(s)) for s in [s1,s2,s3]]

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


[None, None, None]

In [46]:
corpus = data['content'].tolist()
label = data.source.values.tolist()

In [47]:
data.source[:5].values
#备用 y = data.y.values.astype(np.int)

array([0, 0, 0, 1, 0], dtype=object)

In [48]:
corpus[:2]

['此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。\r\nMIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。\r\n当然，关于MIUI 9的确切信息，我们还是等待官方消息。\r\n',
 '骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考虑性能而去屏蔽掉小核心。相反，他们正联手微软，找到一种适合桌面平台的、兼顾性能和功耗的完美方案。\r\n报道称，微软已经拿到了一些新的源码，以便Windows 10更好地理解big.little架构。\r\n资料显示，骁龙835作为一款集成了CPU、GPU、基带、蓝牙/Wi-Fi的SoC，比传统的Wintel方案可以节省至少30%的PCB空间。\r\n按计划，今年Q4，华硕、惠普、联想将首发骁龙835 Win10电脑，预计均是二合一形态的产品。\r\n当然，高通骁龙只是个开始，未来也许还能见到三星Exynos、联发科、华为麒麟、小米澎湃等进入Windows 10桌面平台。\r\n']

In [140]:
label[:5]

[0, 0, 0, 1, 0]

In [145]:
data.source.values.shape

(87052,)

In [146]:
len(label)

87052

### 2.  match regular expression, token and drop stopwords

In [148]:
!pip install tqdm

In [152]:
# Test tqdm
from tqdm import tqdm
from time import sleep

for i in tqdm(range(100), position=0): # 附：这种迭代方式放入正则匹配列表推导式慢了50倍
    sleep(0.01)

100%|██████████| 100/100 [00:01<00:00, 89.80it/s]

In [88]:
corpus[:2]

['此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。\r\nMIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。\r\n当然，关于MIUI 9的确切信息，我们还是等待官方消息。\r\n',
 '骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考虑性能而去屏蔽掉小核心。相反，他们正联手微软，找到一种适合桌面平台的、兼顾性能和功耗的完美方案。\r\n报道称，微软已经拿到了一些新的源码，以便Windows 10更好地理解big.little架构。\r\n资料显示，骁龙835作为一款集成了CPU、GPU、基带、蓝牙/Wi-Fi的SoC，比传统的Wintel方案可以节省至少30%的PCB空间。\r\n按计划，今年Q4，华硕、惠普、联想将首发骁龙835 Win10电脑，预计均是二合一形态的产品。\r\n当然，高通骁龙只是个开始，未来也许还能见到三星Exynos、联发科、华为麒麟、小米澎湃等进入Windows 10桌面平台。\r\n']

In [157]:
# 1:正则匹配(去标点及其余符号) 2:token分词 3:去停用词(先不去停)
corpus_1 = []
corpus_2 = []
#tadm参数：unit-单位,position-(可能因为是多线程故多行出现而非单行bar)
for a in tqdm(corpus, disable=False, unit='it', 
              bar_format='{l_bar}{bar}{r_bar}', 
              position=0):
    corpus_1.append(re.findall(r'\w+', a)) # 1.正则匹配
    article_re = ''.join(re.findall(r'\w+', a))
    corpus_2.append(' '.join(jieba.cut(article_re))) # 2.token分词

100%|██████████| 87052/87052 [01:56<00:00, 745.20it/s] 

In [160]:
len(corpus_2)

87052

In [161]:
corpus_2[0]

'此外 自 本周 6 月 12 日起 除 小米 手机 6 等 15 款 机型 外 其余 机型 已 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 以 确保 工程师 可以 集中 全部 精力 进行 系统优化 工作 有人 猜测 这 也 是 将 精力 主要 用到 MIUI9 的 研发 之中 MIUI8 去年 5 月 发布 距今已有 一年 有余 也 是 时候 更新换代 了 当然 关于 MIUI9 的 确切 信息 我们 还是 等待 官方消息'

In [166]:
print(type(corpus_2))
corpus_2[1]

<class 'list'>


'骁龙 835 作为 唯一 通过 Windows10 桌面 平台 认证 的 ARM 处理器 高通 强调 不会 因为 只 考虑 性能 而 去 屏蔽掉 小 核心 相反 他们 正 联手 微软 找到 一种 适合 桌面 平台 的 兼顾 性能 和 功耗 的 完美 方案 报道 称 微软 已经 拿到 了 一些 新 的 源码 以便 Windows10 更好 地 理解 biglittle 架构 资料 显示 骁龙 835 作为 一款 集成 了 CPUGPU 基带 蓝牙 WiFi 的 SoC 比 传统 的 Wintel 方案 可以 节省 至少 30 的 PCB 空间 按计划 今年 Q4 华硕 惠普 联想 将 首发 骁龙 835Win10 电脑 预计 均 是 二合一 形态 的 产品 当然 高通 骁龙 只是 个 开始 未来 也许 还 能 见到 三星 Exynos 联发科 华为 麒麟 小米 澎湃 等 进入 Windows10 桌面 平台'

### 3. Text Vectorization

In [290]:
# Examples
from sklearn.feature_extraction.text import TfidfVectorizer
corpus_e = [
    '这是 一篇 文档',
    '这是 第二篇 文档',
    '这是 第三篇 文档',
    '这是 第一篇 文档 吗',
]
vectorizer_e = TfidfVectorizer()
X_e = vectorizer_e.fit_transform(corpus_e)
print(vectorizer_e.get_feature_names())

print('(n_samples, n_features): {}'.format(X_e.shape))
print(X_e)
print(type(X_e))

['一篇', '文档', '第一篇', '第三篇', '第二篇', '这是']
(n_samples, n_features): (4, 6)
  (0, 5)	0.41988017955138635
  (0, 0)	0.8046124965719778
  (0, 1)	0.41988017955138635
  (1, 5)	0.41988017955138635
  (1, 1)	0.41988017955138635
  (1, 4)	0.8046124965719778
  (2, 5)	0.41988017955138635
  (2, 1)	0.41988017955138635
  (2, 3)	0.8046124965719778
  (3, 5)	0.41988017955138635
  (3, 1)	0.41988017955138635
  (3, 2)	0.8046124965719778
<class 'scipy.sparse.csr.csr_matrix'>


max_features : int or None (default=None)
If not None, build a vocabulary that only consider the top max_features ordered by term frequency(只考虑tfidf值靠前的) across the corpus.

In [291]:
vectorizer = TfidfVectorizer(max_features=300) # 不加m_f,n_features是268555
X = vectorizer.fit_transform(corpus_2)

In [292]:
X.shape

(87052, 300)

In [293]:
y = label

### 4. Train Set and Test Set Splitting

In [286]:
from sklearn.model_selection import train_test_split

In [288]:
X_train, x_test, Y_train, y_test = train_test_split(X, y)

In [289]:
X_train.shape, x_test.shape

((65289, 300), (21763, 300))

##### KNN
1. KNeighborsClassifier, k=default(5)

In [240]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

In [241]:
x_train, x_valid, y_train, y_valid = train_test_split(X_train, Y_train, random_state=41, test_size=0.15)

In [242]:
x_train.shape, x_valid.shape

((55495, 300), (9794, 300))

In [245]:
knnc = KNeighborsClassifier(n_jobs=-1)

In [246]:
knnc.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform')

In [247]:
knnc.score(x_valid, y_valid)

0.9142332040024505

In [248]:
y_pred = knnc.predict(x_valid)

Returns:	
p : array of shape = [n_samples, n_classes], or a list of n_outputs
of such arrays if n_outputs > 1. The class probabilities of the input samples. 【Classes are ordered by lexicographic(字母、辞典) order.】

In [272]:
y_pred_prob = knnc.predict_proba(x_valid)

In [273]:
y_pred.shape, y_pred_prob.shape

((9794,), (9794, 2))

The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.

In [251]:
precision_score(y_valid, y_pred)

0.9536942458290772

The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.

In [252]:
recall_score(y_valid, y_pred)

0.9511035653650255

Compute the F1 score, also known as balanced F-score or F-measure

The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0. The relative contribution of precision and recall to the F1 score are equal. The formula for the F1 score is:

F1 = 2 * (precision * recall) / (precision + recall)

In [271]:
y_pred_prob

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.

Note: this implementation is restricted to the binary classification task or multilabel classification task in label indicator format.

y_true : array, shape = [n_samples] or [n_samples, n_classes]
True binary labels or binary label indicators.

y_score : array, shape = [n_samples] or [n_samples, n_classes]
Target scores, can either be probability estimates of the 【positive class】, confidence values, or non-thresholded measure of decisions (as returned by “decision_function” on some classifiers). For binary y_true, y_score is supposed to be the score of the class with greater label.

In [303]:
roc_auc_score(y_valid, y_pred_prob[:,1]) # probability estimates of the 【positive class】

0.8986561663748435

1. 使用grid-search确定超参数 Hyper-parameter optimizers¶

In [304]:
from sklearn.model_selection import GridSearchCV

In [327]:
parameters = {'n_neighbors':[3,5,7]}

weights : str or callable, optional (default = ‘uniform’)
weight function used in prediction. Possible values:

‘uniform’ : uniform weights. All points in each neighborhood are weighted equally.
‘distance’ : weight points by the inverse of their distance. 【in this case, closer neighbors of a query point will have a greater influence than neighbors which are further away.】
[callable] : a user-defined function which accepts an array of distances, and returns an array of the same shape containing the weights.

In [328]:
knc = KNeighborsClassifier(n_jobs=5, weights='distance')

In [329]:
# Need a valid scoring value for parameter 'scoring'
# Use sorted(sklearn.metrics.SCORERS.keys()) to get valid options.
import sklearn
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'brier_score_loss',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'mutual_info_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'v_measure_score']

网格搜索超参数未实现，回去看Hands on ML

In [ ]:
clf = GridSearchCV(knc, parameters,cv=5)

In [ ]:
from fastprogress import master_bar, progress_bar
from time import sleep
mb = master_bar(range(1)) # 进度条个数
for i in mb:
    for j in progress_bar(range(100), parent=mb):
        clf.fit(X_train, Y_train)
        mb.child.comment = f'second bar stat'
    mb.first_bar.comment = f'first bar stat'
    mb.write(f'Finished loop {i}.')
    #mb.update_graph(graphs, x_bounds, y_bounds)

In [355]:
# Examples

from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
iris = datasets.load_iris()
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = svm.SVC(gamma="scale")
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(iris.data, iris.target)

sorted(clf.cv_results_.keys())

['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'mean_train_score',
 'param_C',
 'param_kernel',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split0_train_score',
 'split1_test_score',
 'split1_train_score',
 'split2_test_score',
 'split2_train_score',
 'split3_test_score',
 'split3_train_score',
 'split4_test_score',
 'split4_train_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score',
 'std_train_score']

##### Naive Bayes

In [358]:
from sklearn.naive_bayes import MultinomialNB

In [356]:
x_train, x_valid, y_train, y_valid = train_test_split(X_train,Y_train,random_state=42,test_size=0.15)

In [360]:
mnb = MultinomialNB()
mnb.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [362]:
y_pred = mnb.predict(x_valid)
y_pred_proba = mnb.predict_proba(x_valid)

In [364]:
mnb.score(x_valid, y_valid)

0.8910810810810811

In [365]:
precision_score(y_valid, y_pred)

0.9533976596181483

In [366]:
recall_score(y_valid, y_pred)

0.924823259982077

In [367]:
f1_score(y_valid, y_pred)

0.9388931008339652

In [368]:
roc_auc_score(y_valid, y_pred_prob[:,1])

0.8986561663748435

##### Logistic Regression

In [370]:
from sklearn.linear_model import LogisticRegression

In [371]:
lr = LogisticRegression()
lr.fit(x_train, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [372]:
lr.score(x_valid, y_valid)

0.9762162162162162

In [375]:
lr_pred = lr.predict(x_valid)
lr_pred_prob = lr.predict_proba(x_valid)

In [382]:
precision_score(y_valid, y_pred)

0.9533976596181483

In [383]:
recall_score(y_valid, y_pred)

0.924823259982077

In [384]:
f1_score(y_valid, y_pred)

0.9388931008339652

In [385]:
roc_auc_score(y_valid, y_pred_prob[:,1])

0.8986561663748435

### select final model using test set

In [387]:
knnc_score = knnc.score(x_test, y_test)

In [388]:
knnc_score

0.9344463164343697

In [389]:
y_pred = knnc.predict(x_test)
y_pred_prob = knnc.predict_proba(x_test)

In [391]:
precision_score(y_test, y_pred)

0.9632228610077782

In [392]:
recall_score(y_test, y_pred)

0.9643643135263247

In [394]:
f1_score(y_test, y_pred)

0.9637932493020894

In [395]:
roc_auc_score(y_test, y_pred_prob[:,1])

0.9412767537778766

KNN速度慢，precision recall f1均较高 roc_auc 差不多

In [396]:
mnb.score(x_test, y_test)

0.8964619390412008

In [400]:
y_pred = mnb.predict(x_test)
y_pred_proba = mnb.predict_proba(x_test)

In [401]:
precision_score(y_test, y_pred)

0.9551862164984337

In [402]:
recall_score(y_test, y_pred)

0.9291518537328594

In [403]:
f1_score(y_test, y_pred)

0.9419891873337338

In [405]:
roc_auc_score(y_test, y_pred_prob[:,1])

0.9412767537778766

multi-nomial naive bayes速度快，precision较高 recall较低 f1一般 roc_auc差不多

In [407]:
lr.score(x_test, y_test)

0.9760300199111656

In [408]:
y_pred = lr.predict(x_test)
y_pred_prob = lr.predict_proba(x_test)

In [409]:
precision_score(y_test, y_pred)

0.9820605247715651

In [410]:
recall_score(y_test, y_pred)

0.9916201117318436

In [411]:
f1_score(y_test, y_pred)

0.9868171671650592

In [412]:
roc_auc_score(y_test, y_pred_prob[:,1])

0.990005284209644

LR各项指标都挺高

# 抄袭判别(预测为1——新华社，实际为0)

In [414]:
data[:5]

,content,source,ori_source
0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,0,快科技@http://www.kkj.cn/
1,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,0,快科技@http://www.kkj.cn/
2,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,0,快科技@http://www.kkj.cn/
3,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,1,新华社
4,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,0,深圳大件事


In [418]:
y_pred = lr.predict(X)

In [419]:
y_pred.shape

(87052,)

In [422]:
print(type(y))
len(y)

<class 'list'>


87052

In [431]:
# 假如y_pred y到pd中
data['y_actual'] = y
data['y_pred'] = y_pred

In [427]:
data[:5]

,content,source,ori_source,y_actual,y_pred
0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,0,快科技@http://www.kkj.cn/,0,0
1,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,0,快科技@http://www.kkj.cn/,0,0
2,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,0,快科技@http://www.kkj.cn/,0,0
3,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,1,新华社,1,1
4,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,0,深圳大件事,0,0


In [433]:
# select actual==0 but pred==1
copy_contents = data[(data['y_actual']==0) & (data['y_pred']==1)]

In [434]:
copy_contents[:3]

,content,source,ori_source,y_actual,y_pred
20,原标题：叙利亚被“袭机”事件惹怒俄罗斯 警告将瞄准美战机\r\n 海外网6月19日电 ...,0,泡泡网,0,1
51,戈壁的大漠黄沙曾掩埋了无数西域古道，而如今一条大漠天路正顽强地与黄沙“搏斗”，在乌兰布和、腾...,0,新华网,0,1
56,很快，不少人主动添加记者为好友，询问是否需要扫描软件，并声称这些扫描软件能够攻破摄像头的IP...,0,央视新闻,0,1


In [435]:
print('怀疑抄袭数：{}'.format(len(copy_contents)))

怀疑抄袭数：1324


# (Optional and next...)find copy words in copy_contents using edit distance that in 4th)